In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import argparse
from node2vec import Node2Vec

In [ ]:
def read_dimacs_cnf(filename):
    clauses = []
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith("c"):
                continue
            if line.startswith("p cnf"):
                num_vars, num_clauses = map(int, line.strip().split()[2:])
            else:
                clause = list(map(int, line.strip().split()[:-1]))
                clauses.append(clause)
    return num_vars, clauses

def visualize_graph(clauses):
    G = nx.Graph()
    for clause in clauses:
        for literal in clause:
            G.add_node(abs(literal))  # Dodanie węzłów dla zmiennych
        for i in range(len(clause)):
            for j in range(i+1, len(clause)):
                G.add_edge(abs(clause[i]), abs(clause[j]))  # Dodanie krawędzi między zmiennymi w tej samej klauzuli
    pos = nx.spring_layout(G)  # Ustalenie układu wizualizacji
    nx.draw(G, pos, with_labels=True, node_size=500)
    plt.show()

def generate_node_embeddings(clauses):
    G = nx.Graph()
    for clause in clauses:
        for literal in clause:
            G.add_node(abs(literal))
        for i in range(len(clause)):
            for j in range(i+1, len(clause)):
                G.add_edge(abs(clause[i]), abs(clause[j]))

    node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    return model, G

In [ ]:
nazwa_pliku = "DIMACS_files/turbo_easy/example.cnf"
nazwa_pliku_1 = "DIMACS_files/turbo_easy/example_1.cnf"

num_vars, clauses = read_dimacs_cnf(nazwa_pliku)
num_vars_1, clauses_1 = read_dimacs_cnf(nazwa_pliku_1)
print(num_vars)
print(clauses)
print(num_vars_1)
print(clauses_1)

In [ ]:
model, G = generate_node_embeddings(clauses)
node_1_embedding = model.wv.get_vector(str(1))
print("Embedding for node 1:", node_1_embedding)

In [ ]:
node_2_embedding = model.wv.get_vector(str(4))
print("Embedding for node 2:", node_2_embedding)

In [ ]:
for node, _ in model.wv.most_similar('4'):
    print((node, _))

In [ ]:
nx.draw_networkx(G)

In [ ]:
model_1, G1 = generate_node_embeddings(clauses_1)

In [ ]:
for node, _ in model_1.wv.most_similar('1'):
    print((node, _))

In [ ]:
nx.draw_networkx(G1)

In [ ]:
node_IDs = model.wv.index_to_key  # list of node IDs
node_embeddings = model.wv.vectors
print(node_IDs)
print(node_embeddings)

In [ ]:
nx.info(G)

In [ ]:
# nx.draw_networkx(G1)
nx.draw_networkx(G1)

In [ ]:
# G.nodes
for i in G.nodes:
    for node, _ in model.wv.most_similar(f'{i}'):
        print((i, node, _))
    print("-----")

In [ ]:
for i in sorted(G1.nodes):
    print((i, model_1.wv.most_similar(f'{i}')[0]))
    # print("-----")

In [ ]:
def utworzSkierowanyGraf(G1, model_2):
    G2 = nx.DiGraph()
    for i in sorted(G1.nodes):
        node, waga = model_2.wv.most_similar(f'{i}')[0]
        # print(i, int(node), round(waga,2))
        G2.add_edge(i, int(node))  # Dodanie krawędzi z wagą 5
    # G2.add_edge(2, 3, weight=3)  # Dodanie krawędzi z wagą 3
    # G2.add_edge(3, 4, weight=7)  # Dodanie krawędzi z wagą 7

    # Rysowanie grafu
    # pos = nx.shell_layout(G2)
    pos = nx.spring_layout(G2, scale=10)
    # pos = nx.kamada_kawai_layout(G2)
    nx.draw(G2, pos, with_labels=True, node_size=150, arrowsize=20)
    # Dodanie etykiet krawędzi
    labels = nx.get_edge_attributes(G2, 'weight')
    nx.draw_networkx_edge_labels(G2, pos, edge_labels=labels)
    plt.show()

In [ ]:
utworzSkierowanyGraf(G, model)
utworzSkierowanyGraf(G1, model_1)

In [ ]:
def cala_operacja(nazwa_pliku):
    num_vars, clauses = read_dimacs_cnf(nazwa_pliku)
    model, G = generate_node_embeddings(clauses)
    utworzSkierowanyGraf(model, G)

nazwa = "DIMACS_files/turbo_easy/example_1.cnf"
cala_operacja(nazwa)